In [1]:
from ovcg import *
from fesol import *
import numpy as np
import scipy.io as sio
from interface import DynamicUnderRelaxation, InterfaceData, RelativeCovergenceMonitor

In [2]:
k=20
# problem discription:
# basically, with a fixed Pr=0.01 and Re=500, the k is the ratio between
# kappa_s/kappa_f, where kappa stands for conductivity coefficient.
assert k == 1 or k == 5 or k == 20

In [3]:
# since this is a matching interface, we can create
# exact interface nodes mappings.
# this step is done offline
inodes = 100
s2f = np.zeros(inodes, dtype=int)
f2s = s2f.copy()
s2f[1] = inodes-1
start = inodes-2
for i in range(inodes-2):
    s2f[i+2] = start
    start = start - 1
count = 0
for i in range(inodes):
    f2s[s2f[i]] = i

In [4]:
# fluid solver
interface_grid = 3
interface_side = 0
interface_axis = 1

solverF = Cgins()
solverF.read_cmd('fluid_inputs/fluid_flat_plate%i.cmd' % k)
solverF.init_grid()
solverF.init_temperature_interface(
    grid=interface_grid,
    side=interface_side,
    axis=interface_axis
)
solverF.init_solver(debug=0, logname='fluid_outputs/dummy')
fluid_ofile = VtkFile('fluid_outputs/resFDSR%i.pvd' % k)

In [5]:
# solid solver
fixed_temp_tag = 2
adiabatic_tag = 3
interface_tag = 4

kappa = 100.0
rho = 1.0
Cp = 100.0
T_bot = 310.0

solverS = HeatSolver(kappa=kappa, rho=rho, Cp=Cp, transient=True)
solverS.output_file = 'solid_outputs/resFDSR%i.pvd' % k
solverS.load_mesh('solid_inputs/solid_plate.msh')
solverS.set_initial_condition(T_bot)
solverS.define_adiabatic_bd(tag=adiabatic_tag)
solverS.define_const_temperature_bd(tag=fixed_temp_tag, value=T_bot)
solverS.define_robin_interface(tag=interface_tag)
solverS.init_solver()

In [6]:
# coupling data and setups
abTempF = InterfaceData(size=inodes, value=300.0)
abTempF2S = InterfaceData(size=inodes, value=300.0)
TempS = InterfaceData(size=inodes, value=T_bot)
tempF = InterfaceData(size=inodes, value=300.0)

under_relax = DynamicUnderRelaxation(init_omega=1.0)
conv_mntr = RelativeCovergenceMonitor(tol=1e-6)

# maximum pc steps allowed
max_pc_steps = 200

# since this problem has no mesh motion and both kappa are material properties
# so the heat transfer coefficients don't vary in time
hF2S = solverF.get_heat_transfer_coeff(
    grid=interface_grid,
    side=interface_side,
    axis=interface_axis
)[f2s]

In [7]:
# coupling timing information, use fixed time step for now
Tfinal = 1.0
N = 100
dt = Tfinal/N
t = 0.0
avg_pc_iterations = 0
flog = open('FDSR%i.log' % k, mode='w')
flog.write('Fluid Dirichlet with solid Robin setting, study case: %i\n' % k)
flog.close()
flog = open('FDSR%i.log' % k, mode='a')

In [8]:
# begin to couple
for step in range(N):
    t += dt
    
    pc_counts = 0
    
    # back up solutions
    solverF.backup_current_solutions()
    solverS.backup()
    
    while True:
        pc_counts = pc_counts + 1
        
        # back up previous interface value
        tempF.backup()
        
        # advance fluid
        solverF.advance(t=t, dt=dt)
        # retrieve fluid ambient temperature
        abTempF.curr[:] = solverF.get_ambient_temperature(
            grid=interface_grid,
            side=interface_side,
            axis=interface_axis
        )
        
        # retrieve the wall temperature for convergence testing
        tempF.curr[:] = solverF.get_temperature_interface(
            grid=interface_grid,
            side=interface_side,
            axis=interface_axis
        )
        
        # mapping
        abTempF2S.curr[:] = abTempF.curr[f2s]
        # update solid flux interface
        solverS.update_robin_interface(
            tag=interface_tag,
            Ta=abTempF2S.curr,
            h_coeff=hF2S
        )
        
        # advance solid
        solverS.advance(t=t, dt=dt)
        # retrieve solid interface temperature
        TempS.curr[:] = solverS.get_interface_temperature(tag=interface_tag)
        
        # mapping
        tempF.curr[:] = TempS.curr[s2f]
        
        # update residual
        tempF.update_res()
        
        # test convergence
        is_conv = conv_mntr.determine_convergence(tempF)
        if is_conv or pc_counts >= max_pc_steps:
            break
        else:
            # if not converge, then underrelaxation and update to fluid then restore
            under_relax.determine_omega(tempF)
            under_relax.update_solution(tempF)
            solverF.put_temperature_interface(
                grid=interface_grid,
                side=interface_side,
                axis=interface_axis,
                data=tempF.curr
            )
            solverF.restore_previous_solutions()
            solverS.restore()  

    # outputs
    msg = 'step=%i, pc_iterations=%i.' % (step, pc_counts)
    print(msg)
    flog.write(msg+'\n')
    solverS.write()
    fluid_ofile.write(solverF, t)
    avg_pc_iterations = avg_pc_iterations + pc_counts
flog.close()

step=0, pc_iterations=20.
step=1, pc_iterations=14.
step=2, pc_iterations=11.
step=3, pc_iterations=8.
step=4, pc_iterations=10.
step=5, pc_iterations=11.
step=6, pc_iterations=11.
step=7, pc_iterations=11.
step=8, pc_iterations=11.
step=9, pc_iterations=11.
step=10, pc_iterations=11.
step=11, pc_iterations=11.
step=12, pc_iterations=10.
step=13, pc_iterations=10.
step=14, pc_iterations=10.
step=15, pc_iterations=10.
step=16, pc_iterations=10.
step=17, pc_iterations=10.
step=18, pc_iterations=9.
step=19, pc_iterations=9.
step=20, pc_iterations=9.
step=21, pc_iterations=9.
step=22, pc_iterations=9.
step=23, pc_iterations=9.
step=24, pc_iterations=9.
step=25, pc_iterations=9.
step=26, pc_iterations=8.
step=27, pc_iterations=8.
step=28, pc_iterations=8.
step=29, pc_iterations=8.
step=30, pc_iterations=8.
step=31, pc_iterations=8.
step=32, pc_iterations=8.
step=33, pc_iterations=8.
step=34, pc_iterations=8.
step=35, pc_iterations=8.
step=36, pc_iterations=8.
step=37, pc_iterations=7.
step=

In [9]:
theta=(solverF.get_temperature_interface(3,0,1)-300)/10
print(theta)
sio.savemat('numeric_res/kFDSR%i.mat' % k, {'theta': theta, 'avg_pc_its': avg_pc_iterations/N})

[0.91189655 0.91350647 0.91602824 0.91842757 0.92064839 0.92259909
 0.92438484 0.92600075 0.92748501 0.92889326 0.93018089 0.93138982
 0.93251132 0.93356932 0.9345532  0.93548068 0.93634735 0.93716822
 0.93793759 0.93865855 0.93933787 0.93997505 0.94057704 0.94114039
 0.94166887 0.94216421 0.94263209 0.94306967 0.94348171 0.94386741
 0.94422762 0.94456614 0.94488203 0.94517695 0.94545437 0.94571151
 0.94595136 0.94617373 0.94638186 0.94657404 0.94675219 0.94691748
 0.94706893 0.94720818 0.94733565 0.94745172 0.94755709 0.94765191
 0.94773657 0.9478122  0.94787761 0.9479344  0.94798181 0.9480205
 0.94805087 0.94807205 0.94808574 0.94809031 0.94808676 0.94807458
 0.94805382 0.94802509 0.9479868  0.94793995 0.94788292 0.94781696
 0.94774123 0.94765438 0.94755748 0.94744869 0.94732766 0.94719481
 0.94704769 0.94688772 0.94671178 0.94652017 0.94630965 0.94608229
 0.9458349  0.94556495 0.94527325 0.9449543  0.94460837 0.94423513
 0.94382654 0.9433838  0.94289886 0.94237179 0.94179576 0.94116